# Volt
# Summary 
In this project the National Vulnerability Database's (NVD) of Common exposures and vulnerabilites (CVES) is levarged
to create a tool that is used as a part of the data pipeline to determine how similar CVE's are to one another. 

The first goal to such a pipeline is to clean the description data, tokenize the data. The second goal is tokenize and read the data that will be used as a training label. 
## Project Objective: 
### Create a Pipeline that Can Support Document Similarity and Search


# Requirements
Required packages:
- pyspark
- numpy

TF-IDF use cases:
- LDA
- Similarity with
1. **Feature Selection**:
   - Examine the top TF-IDF terms to identify important features. You can select a subset of these features for further analysis or modeling.
   - Consider removing low-TF-IDF terms (common words) that might not contribute significantly to your task.

2. **Clustering and Topic Modeling**:
   - Apply clustering algorithms (e.g., K-means, DBSCAN) to group similar documents based on their TF-IDF vectors.
   - Explore topic modeling techniques (e.g., Latent Dirichlet Allocation, Non-Negative Matrix Factorization) to discover latent topics within your corpus.

3. **Document Similarity**:
   - Calculate cosine similarity between TF-IDF vectors of different documents. This helps identify similar documents.
   - Use similarity scores to recommend related articles, products, or content.

4. **Classification and Sentiment Analysis**:
   - Train classifiers (e.g., SVM, Random Forest) using TF-IDF features as input. This is useful for tasks like sentiment analysis, spam detection, or document categorization.
   - Convert text data into TF-IDF vectors and use them as features for machine learning models.

5. **Search and Information Retrieval**:
   - Build an inverted index using TF-IDF vectors to create an efficient search engine.
   - Retrieve relevant documents based on user queries by ranking them using their TF-IDF scores.

6. **Visualizations**:
   - Visualize TF-IDF scores using word clouds, scatter plots, or bar charts to gain insights into term importance.
   - Plot the distribution of TF-IDF values across the entire dataset.

7. **Optimize Hyperparameters**:
   - Experiment with different parameters (e.g., n-grams, stop words, max features) in your TF-IDF vectorization process.
   - Use cross-validation to find optimal settings.

TODO: Find sources to back up these claims.

In [119]:
import os, math, re
import pyspark
import urllib.request
import zipfile
from pyspark.sql import SparkSession
from pyspark.sql.functions import ascend, col, count, countDistinct, concat_ws, desc, explode, expr, lit, udf, split, sum
from pyspark.sql.types import DoubleType, FloatType, StringType, ArrayType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.linalg import VectorUDT, SparseVector

ImportError: cannot import name 'ascend' from 'pyspark.sql.functions' (/home/rsandor/.local/lib/python3.9/site-packages/pyspark/sql/functions.py)

# Download data
Handle downloading the data if it doesn't already exist for the user.

In [76]:
# Load the dataset if it doesn't already exist. 
data_dir = "data"
os.makedirs(data_dir, exist_ok=True) 
fileUrls = [
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2002.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2003.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2004.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2005.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2006.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2007.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2008.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2009.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2010.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2011.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2012.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2013.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2014.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2015.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2016.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2017.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2018.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2019.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2020.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2021.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2022.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2023.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2024.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-recent.json.zip'
    ]

# Iterate through each URL
for url in fileUrls:
    filename = url.split("/")[-1]
    outputfile = os.path.join(data_dir, filename)
    checkfile = os.path.join(data_dir, os.path.splitext(filename)[0])

    # Check if the file already exists
    if not os.path.exists(checkfile):
        # Download the file
        urllib.request.urlretrieve(url, outputfile)
        print(f"Downloaded: {filename}")

        # Extract the file
        with zipfile.ZipFile(outputfile, "r") as zip_ref:
            zip_ref.extractall(data_dir)

        # Delete the original zip file
        os.remove(outputfile)

In [77]:
spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("voltcve")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .config("spark.default.parallelism", 8)
        .config("spark.driver.memory", "25g") \
       .config("spark.executor.memory", "10g") \
        .getOrCreate()
)
# read the data
cves = spark.read.option("multiline", "true").json("data/nvdcve-1.1-2020.json")

# Manipulate the data to be more usable 
cves.printSchema()


root
 |-- CVE_Items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- configurations: struct (nullable = true)
 |    |    |    |-- CVE_data_version: string (nullable = true)
 |    |    |    |-- nodes: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |    |    |-- cpe_match: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |    |-- cpe23Uri: string (nullable = true)
 |    |    |    |    |    |    |    |    |    |-- cpe_name: array (nullable = true)
 |    |    |    |    |    |    |    |    |    |    |-

# Complex JSON
Look at the Schema above, the data is complex and Spark can read the data into a DataFrame. The DataFrame being equvielent to Spark SQL. 

From the complex schema the data needs to be turned into a more usable format. 

## Section Objective:
- Get Ids
- Description Data
- Remove unnecessary nesting

In [111]:
exploded = cves.select(explode(col("CVE_Items")).alias("cves"))

descr_df = exploded.select(col("cves.cve.CVE_data_meta.ID").alias("id"),
          col("cves.cve.description.description_data.value").alias("description"));

descr_df = descr_df.withColumn("description_single", concat_ws(" ", descr_df["description"]))

In [112]:
doc_count = descr_df.selectExpr("count(distinct id)").first()[0]
print("Number of docs: {}".format(doc_count))

Number of docs: 20453


# CVE Product Info
Below the goal is to get useful information about the CVEs. For example, the type of tool or software that had a vulnerability.
- Scheme format: https://en.wikipedia.org/wiki/Common_Platform_Enumeration
- If it is not used in the application the data may be used in other applications the data later.


Example of CPEs: \
cpe:2.3:a:ntp:ntp:4.2.8:p3:*:*:*:*:*:* \
cpe:2.3:o:microsoft:windows_7:-:sp2:*:*:*:*:*:* \
cpe:2.3:a:microsoft:internet_explorer:8.0.6001:beta:*:*:*:*:*:*



In [113]:
# Now 'exploded' contains individual rows for each 'CVE_Item'
# Scheme format: https://en.wikipedia.org/wiki/Common_Platform_Enumeration
# cpe:<cpe_version>:<part>:<vendor>:<product>:<version>:<update>:<edition>:<language>:<sw_edition>:<target_sw>:<target_hw>:<other>
cpe_df = exploded.select(col("cves.cve.CVE_data_meta.ID").alias("id"), explode(col("cves.configurations.nodes.cpe_match")[0]).alias("cpe"))

cpe_df = cpe_df.select(col("id"), col("cpe").alias("cpe"))
cpe_df = cpe_df.select("id", split(cpe_df.cpe.cpe23uri,":",-1)[4].alias("product"))

# Sample of the data - schema/show
cpe_df.printSchema()
cpe_df.show(n=50, truncate=200)

root
 |-- id: string (nullable = true)
 |-- product: string (nullable = true)



+-------------+-------+
|           id|product|
+-------------+-------+
|CVE-2020-0001|android|
|CVE-2020-0001|android|
|CVE-2020-0001|android|
|CVE-2020-0001|android|
|CVE-2020-0002|android|
|CVE-2020-0002|android|
|CVE-2020-0002|android|
|CVE-2020-0002|android|
|CVE-2020-0003|android|
|CVE-2020-0004|android|
|CVE-2020-0004|android|
|CVE-2020-0004|android|
|CVE-2020-0004|android|
|CVE-2020-0005|android|
|CVE-2020-0005|android|
|CVE-2020-0005|android|
|CVE-2020-0005|android|
|CVE-2020-0006|android|
|CVE-2020-0006|android|
|CVE-2020-0006|android|
|CVE-2020-0006|android|
|CVE-2020-0007|android|
|CVE-2020-0007|android|
|CVE-2020-0007|android|
|CVE-2020-0007|android|
|CVE-2020-0008|android|
|CVE-2020-0008|android|
|CVE-2020-0008|android|
|CVE-2020-0008|android|
|CVE-2020-0009|android|
|CVE-2020-0010|android|
|CVE-2020-0011|android|
|CVE-2020-0012|android|
|CVE-2020-0014|android|
|CVE-2020-0014|android|
|CVE-2020-0014|android|
|CVE-2020-0014|android|
|CVE-2020-0015|android|
|CVE-2020-0015|a

## Uses from the cpe data:
- CPE could be a training label.
- CPE used in clustering.
- Document filtering. 


# Tokenization: 
## Applying the right tokenization methods:
The dataset contains a lot of data the ordinary tokenization may not apply. For example, file names contain puncutation and can leave the token meaningless. 
An effort was made to preserve all meaninful punction that would describe software or hardware configurations, while removing stop words, and ordinary english punctation. 

In [192]:
@udf
def string_cleaner(input_str):
    # 1. Replace all "." or ':' followed by whitespace with an empty string.
    # a. Remove ending periods.
    # 2. Remove trademark, rights.
    # 3. Grab cotent in parentheses only.
    # 4. Remove some punctuation.
    cleaned_text = re.sub(r"[.:,]+\s+", " ", input_str)
    # Remove trailing periods
    cleaned_text = re.sub(r"\.$", "", cleaned_text)
    # Remove apostrophes, (TM), (R), parentheses, and double quotes, newlines
    cleaned_text = re.sub(r"\'|\(TM\)|\(R\)|\(|\)|\"|[\n]+", "", cleaned_text)
    # Convert to lowercase and strip leading/trailing spaces
    cleaned_text = cleaned_text.lower().strip()
    return cleaned_text

clean_tokens = descr_df.withColumn("token", split(string_cleaner(col("description_single")), " " ))
stop_words_remover = StopWordsRemover(inputCol="token", outputCol="cleanToken")
clean_tokens = stop_words_remover.transform(clean_tokens)
clean_tokens = clean_tokens.select("id", explode("cleantoken").alias("token"))
clean_tokens = clean_tokens.filter(col("token").isNotNull())
# Remove words that are only one character
# https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
clean_tokens = clean_tokens.rdd.filter(lambda x: x['token'] != "")
clean_tokens = clean_tokens.filter(lambda x: len(x['token']) > 2).toDF()
clean_tokens.show()
# # Show the resulting DataFrame
clean_tokens.show(truncate=200)

+-------------+--------------------+
|           id|               token|
+-------------+--------------------+
|CVE-2020-0001|getprocessrecordl...|
|CVE-2020-0001|activitymanagerse...|
|CVE-2020-0001|            isolated|
|CVE-2020-0001|                apps|
|CVE-2020-0001|             handled|
|CVE-2020-0001|           correctly|
|CVE-2020-0001|                lead|
|CVE-2020-0001|               local|
|CVE-2020-0001|          escalation|
|CVE-2020-0001|           privilege|
|CVE-2020-0001|          additional|
|CVE-2020-0001|           execution|
|CVE-2020-0001|          privileges|
|CVE-2020-0001|              needed|
|CVE-2020-0001|                user|
|CVE-2020-0001|         interaction|
|CVE-2020-0001|              needed|
|CVE-2020-0001|        exploitation|
|CVE-2020-0001|             product|
|CVE-2020-0001|             android|
+-------------+--------------------+
only showing top 20 rows



+-------------+---------------------------+
|           id|                      token|
+-------------+---------------------------+
|CVE-2020-0001|     getprocessrecordlocked|
|CVE-2020-0001|activitymanagerservice.java|
|CVE-2020-0001|                   isolated|
|CVE-2020-0001|                       apps|
|CVE-2020-0001|                    handled|
|CVE-2020-0001|                  correctly|
|CVE-2020-0001|                       lead|
|CVE-2020-0001|                      local|
|CVE-2020-0001|                 escalation|
|CVE-2020-0001|                  privilege|
|CVE-2020-0001|                 additional|
|CVE-2020-0001|                  execution|
|CVE-2020-0001|                 privileges|
|CVE-2020-0001|                     needed|
|CVE-2020-0001|                       user|
|CVE-2020-0001|                interaction|
|CVE-2020-0001|                     needed|
|CVE-2020-0001|               exploitation|
|CVE-2020-0001|                    product|
|CVE-2020-0001|                 

# Improvements
In previous iterations of the application it became apparent that this dataset has its own set of stop words that causes TF-IDF Vectors (show in latter section), 
to have matches that were not as good as they could because of similar low importance words. 

Given that this is a dataset about CVEs words like 'attack', 'exploit', 'vulnerab*', don't convey as much meaning.  

For example examine the top words after removing common stop words: 

In [193]:
clean_tokens.groupBy("token").agg(countDistinct("id").alias("df")).orderBy(desc("df")).show(n=50)

+-------------+----+
|        token|  df|
+-------------+----+
|vulnerability|8523|
|     attacker|6020|
|       allows|5608|
|          via|4681|
|       remote|3847|
|     versions|3713|
|         user|3690|
|         code|3632|
|        issue|3367|
|    arbitrary|3346|
|       access|3181|
|     affected|2809|
|      crafted|2628|
|        allow|2548|
|    attackers|2485|
|    execution|2450|
|          use|2434|
|          may|2407|
|         file|2332|
|      service|2264|
|       exists|2222|
|       system|2182|
|  information|2104|
|      exploit|2091|
|         data|2035|
|      execute|2023|
|   discovered|1897|
|      version|1862|
|        prior|1812|
|          xss|1796|
|          due|1783|
|   successful|1778|
|       server|1755|
|        cause|1736|
|       number|1718|
|        local|1684|
|       memory|1663|
|         lead|1622|
|       reason|1614|
|     rejected|1612|
|authenticated|1611|
|   privileges|1591|
|    malicious|1582|
|       denial|1575|
|        note

Lets resolve some of these:

In [194]:
stopwordslist=['attack', 'attacks', 'attacker', 'attackers',  'vulnerability', 'due', 'may', '1','exploit', 'affects', 'affected', 'exists', 'version', 'versions', 'id' ]
clean_tokens = clean_tokens.filter(~col("token").isin(*stopwordslist))
clean_tokens = clean_tokens.groupBy("token").agg(countDistinct("id").alias("df")).orderBy("token")
# Only include the tokens which occur in more than one document. clean_tokens = clean_tokens.filter(clean_tokens['df'] > 1)
clean_tokens.cache()

clean_tokens.show(n=50)


+--------------------+---+
|               token| df|
+--------------------+---+
|  !j@l#y$z%x6x7q8c9z|  2|
|                 #gp|  2|
|               $_get|  2|
|              $_post|  3|
|                 %0a|  2|
|              %path%|  3|
|%programfiles%\1e...|  2|
|               &amp;|  3|
|        &quot;public|  2|
|          &quot;safe|  2|
|             **not**|  2|
|        **resolved**|  2|
|           **version|  5|
|       *.example.com|  2|
|               *ctxt|  2|
|               *note| 19|
|                 *pb|  2|
|                 -->|  4|
|            --config|  2|
|--enable-experime...|  3|
|            --output|  2|
|              -dsafe|  2|
|                 -s+|  5|
|                 ...|  2|
|                 ../| 20|
|../../programs/dw...|  2|
|                 ..\|  3|
|             .append|  2|
|                .bat|  2|
|                .bmp|  2|
|                .bss|  2|
|                 .cf|  3|
|                .dll|  4|
|               .docx|  4|
|

In [181]:
clean_tokens.select(countDistinct("token")).show()
clean_tokens_count = clean_tokens.count()

+---------------------+
|count(DISTINCT token)|
+---------------------+
|                40615|
+---------------------+



In [84]:
def tfidf(tokens, doc_count):
    allTokensForId = tokens.groupBy("id").agg(count("id").alias("allTokensForId"))

    tfds = tokens.groupBy("id", "token").agg(count("id").alias("rawtf"))
    dfds = tokens.groupBy("token").agg(countDistinct("id").alias("df"))

    # Join the two DataFrames on 'id'
    merged_df = tfds.join(allTokensForId, on="id")

    # Calculate the ratio of rawtf to allTokensForId
    tfds = merged_df.withColumn("tf", col("rawtf") / col("allTokensForId"))
    
    merged_df.show()

    # Define the UDF for idf calculation
    spark.udf.register("calcidfudf", lambda df: calcidf(doc_count, df), DoubleType())

    # Calculate idf and add it as a new column "idf"
    tokens_idf = dfds.withColumn("idf", expr("calcidfudf(df)"))

    # Show the resulting dataframe
    tfidfds = tokens_idf.join(tfds, "token", "left") \
        .withColumn("tf_idf", col("tf") * col("idf"))

    return tfidfds

def calcidf(doc_count, df):
    # Calculate the tf-idf using natural log
    return math.log((doc_count + 1.0) / (df + 1.0))


In [85]:
tfidf_df = tfidf(clean_tokens, doc_count)

tfidf_df.show()

+-------------+-----------+-----+--------------+
|           id|      token|rawtf|allTokensForId|
+-------------+-----------+-----+--------------+
|CVE-2020-0002|       code|    1|            27|
|CVE-2020-0003|       user|    1|            25|
|CVE-2020-0005|a-141552859|    1|            26|
|CVE-2020-0006|     needed|    2|            31|
|CVE-2020-0022|       code|    1|            27|
|CVE-2020-0022|  android-9|    1|            27|
|CVE-2020-0029|       lead|    1|            24|
|CVE-2020-0032|     needed|    2|            26|
|CVE-2020-0033|      write|    1|            25|
|CVE-2020-0038|  rw_i93.cc|    1|            27|
|CVE-2020-0082|      local|    1|            24|
|CVE-2020-0082|interaction|    1|            24|
|CVE-2020-0097|interaction|    1|            25|
|CVE-2020-0109|a-148059175|    1|            25|
|CVE-2020-0110|      psi.c|    1|            26|
|CVE-2020-0118|interaction|    1|            23|
|CVE-2020-0138|    missing|    1|            29|
|CVE-2020-0156|a-139

24/03/22 05:47:08 WARN SimpleFunctionRegistry: The function calcidfudf replaced a previously registered function.


+-----------+---+-----------------+--------------+-----+--------------+--------------------+-------------------+
|      token| df|              idf|            id|rawtf|allTokensForId|                  tf|             tf_idf|
+-----------+---+-----------------+--------------+-----+--------------+--------------------+-------------------+
|   11.12.77| 14|7.217883540263748| CVE-2020-0595|    1|            22|0.045454545454545456|0.32808561546653403|
|   11.12.77| 14|7.217883540263748| CVE-2020-0540|    1|            19| 0.05263157894736842| 0.3798886073823025|
|   11.12.77| 14|7.217883540263748| CVE-2020-0538|    1|            20|                0.05|0.36089417701318743|
|   11.12.77| 14|7.217883540263748| CVE-2020-8674|    1|            22|0.045454545454545456|0.32808561546653403|
|   11.12.77| 14|7.217883540263748| CVE-2020-0531|    1|            19| 0.05263157894736842| 0.3798886073823025|
|   11.12.77| 14|7.217883540263748| CVE-2020-0596|    1|            23|0.043478260869565216| 0.3

Already calculated the TF-IDF scores for the documents. Now, let’s create a TF-IDF vector from these scores.

Assuming you have a DataFrame named tfidf_vector with columns ‘id’, ‘token’, ‘tf_idf’, and ‘allTokensForId’, you can proceed as follows:

    Group TF-IDF Scores by Document ID:
        Group the DataFrame by the ‘id’ column and aggregate the ‘tf_idf’ values into a list for each document.
        This will give you a list of TF-IDF scores for each document.

    Create a Sparse Vector Representation:
        Use the SparseVector class from PySpark to create a sparse vector representation for each document.
        The vector will have dimensions equal to the total number of unique tokens (terms) in your dataset.
        For each document, set the value at the index corresponding to the token to its corresponding TF-IDF score.

    Assemble the Sparse Vectors:
        Assemble the sparse vectors into a single column using the VectorAssembler.
        This will give you a new DataFrame with a column containing the TF-IDF vectors.


In [86]:
# This is just to compare how my vectorization method compares to ML libs 
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, Normalizer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import SparseVector, DenseVector
grouped_tfidf = tfidf_df.groupBy('id').agg(collect_list('token').alias('words'))
ht = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20000)
featurizedData = ht.transform(grouped_tfidf)

normalizer = Normalizer(inputCol="rawFeatures", outputCol="normFeatures")
normData = normalizer.transform(featurizedData)

first_doc_features = normData.filter(col('id') == 'CVE-2020-0003').select('normFeatures').first()[0]

# Get the SparkContext from the SparkSession
sc = spark.sparkContext

# Broadcast the first document's features
first_doc_features_bc = sc.broadcast(first_doc_features)

# Define a UDF to compute cosine similarity
def cosine_similarity(v):
    return float(first_doc_features_bc.value.dot(v) / (first_doc_features_bc.value.norm(2) * v.norm(2)))

cosine_similarity_udf = udf(cosine_similarity, DoubleType())

# Compute cosine similarity
result = normData.withColumn("cosine_sim", cosine_similarity_udf(col('normFeatures')))

# Get the top similar documents
top_similar_docs = result.sort(col("cosine_sim").desc()).limit(5)

top_similar_docs.show()

+-------------+--------------------+--------------------+--------------------+------------------+
|           id|               words|         rawFeatures|        normFeatures|        cosine_sim|
+-------------+--------------------+--------------------+--------------------+------------------+
|CVE-2020-0003|[android, interac...|(20000,[348,392,1...|(20000,[348,392,1...|1.0000000000000002|
|CVE-2020-0215|[android, interac...|(20000,[104,392,1...|(20000,[104,392,1...|0.6366550033321677|
|CVE-2020-0009|[a-142938932, cor...|(20000,[392,1662,...|(20000,[392,1662,...|0.6304883249912806|
|CVE-2020-0001|[android-8.1, nee...|(20000,[270,392,2...|(20000,[270,392,2...| 0.595879571531124|
|CVE-2020-0202|[android, develop...|(20000,[392,1662,...|(20000,[392,1662,...|0.5910828046793257|
+-------------+--------------------+--------------------+--------------------+------------------+



In [ ]:
"""
    Create a sparse vector for each of the documents in the dataset filled with 0's,
    replacing the 0 with values that exist for that token on the id.
"""
def create_sparse_vector(document_tf_idf, num_tokens):
    indices = range(num_tokens)
    values = [0.0] * num_tokens

    return SparseVector(num_tokens, indices, values)



In [106]:
from pyspark.sql import Row
tfidf_df.show()

grouped_tfidf = tfidf_df.groupBy('id').agg(collect_list('token').alias('document'),
    collect_list('tf_idf').alias('tfidf_list'))

def sort_words(record):
    id, words, tf_idf = record
    sorted_words = sorted(words)
    sorted_tf_idf = [x for _, x in sorted(zip(words, tf_idf))]
    return Row(id=id, words_list=sorted_words, tfidf_list=sorted_tf_idf)

rdd = grouped_tfidf.rdd.map(sort_words)

grouped_tfidf = rdd.toDF()

grouped_tfidf.show(truncate=200)

+-----------+---+-----------------+--------------+-----+--------------+--------------------+-------------------+
|      token| df|              idf|            id|rawtf|allTokensForId|                  tf|             tf_idf|
+-----------+---+-----------------+--------------+-----+--------------+--------------------+-------------------+
|   11.12.77| 14|7.217883540263748| CVE-2020-0595|    1|            22|0.045454545454545456|0.32808561546653403|
|   11.12.77| 14|7.217883540263748| CVE-2020-0540|    1|            19| 0.05263157894736842| 0.3798886073823025|
|   11.12.77| 14|7.217883540263748| CVE-2020-0538|    1|            20|                0.05|0.36089417701318743|
|   11.12.77| 14|7.217883540263748| CVE-2020-8674|    1|            22|0.045454545454545456|0.32808561546653403|
|   11.12.77| 14|7.217883540263748| CVE-2020-0531|    1|            19| 0.05263157894736842| 0.3798886073823025|
|   11.12.77| 14|7.217883540263748| CVE-2020-0596|    1|            23|0.043478260869565216| 0.3

In [98]:
from pyspark.ml.linalg import SparseVector
from pyspark.sql.functions import collect_list
from pyspark.ml.feature import VectorAssembler

grouped_tfidf = tfidf_df.groupBy('id').agg(collect_list('tf_idf').alias('tfidf_list'))

# Create Sparse Vectors
def create_sparse_vector(tfidf_list, num_tokens):
    indices = range(num_tokens)
    values = [0.0] * num_tokens  # Initialize all values to 0.0
    for i, tfidf_score in enumerate(tfidf_list):
        values[i] = tfidf_score
    return SparseVector(num_tokens, indices, values)

sparse_vector_udf = udf(create_sparse_vector, VectorUDT())

tfidf_vector_with_sparse = grouped_tfidf.withColumn('tfidf_vector', sparse_vector_udf('tfidf_list', lit(clean_tokens_count)))

# Assemble the Sparse Vectors
assembler = VectorAssembler(inputCols=['tfidf_vector'], outputCol='tfidf_features')
final_tfidf_vector = assembler.transform(tfidf_vector_with_sparse)

# Show the resulting DataFrame
final_tfidf_vector.select('id', 'tfidf_features').show(truncate=100)


+-------------+---------------------------------+
|           id|                   tfidf_features|
+-------------+---------------------------------+
|CVE-2020-0001|(40649,[0],[0.33403364866032764])|
|CVE-2020-0001|(40649,[0],[0.23368408229445767])|
|CVE-2020-0001|(40649,[0],[0.21570944722421723])|
|CVE-2020-0001|(40649,[0],[0.11643150796292194])|
|CVE-2020-0001|  (40649,[0],[0.096015880714596])|
|CVE-2020-0002| (40649,[0],[0.3216620320432785])|
|CVE-2020-0002|  (40649,[0],[0.064004433672028])|
|CVE-2020-0002|(40649,[0],[0.11211922989022112])|
|CVE-2020-0003| (40649,[0],[0.3473949946067408])|
|CVE-2020-0003|(40649,[0],[0.12108876828143882])|
|CVE-2020-0003|(40649,[0],[0.09985651594317985])|
|CVE-2020-0004|(40649,[0],[0.10811497167985608])|
|CVE-2020-0004|(40649,[0],[0.08915760352069628])|
|CVE-2020-0005|(40649,[0],[0.35510717541561587])|
|CVE-2020-0005|(40649,[0],[0.11643150796292194])|
|CVE-2020-0005|  (40649,[0],[0.096015880714596])|
|CVE-2020-0006| (40649,[0],[0.2801572537151135])|


Now that you have the TF-IDF vectors for the documents, here are some useful things to do with the results:

    Document Similarity:
        Calculate the similarity between documents using cosine similarity or other distance metrics. The closer the vectors, the more similar the documents.
        For example, you can find similar documents to a given query document by comparing their TF-IDF vectors.

    Topic Modeling:
        Apply topic modeling techniques (such as Latent Dirichlet Allocation or Non-Negative Matrix Factorization) to discover underlying topics in your corpus.
        Use the TF-IDF vectors as input for these models.

    Classification and Clustering:
        Train machine learning models (e.g., SVM, Random Forest, or k-means) using the TF-IDF vectors as features.
        Classify documents into predefined categories or cluster similar documents together.

    Keyword Extraction:
        Identify important keywords or phrases within each document based on their TF-IDF scores.
        Higher TF-IDF scores indicate more significant terms.

    Search and Retrieval:
        Use the TF-IDF vectors to build an efficient search index for your documents.
        Given a query, retrieve relevant documents based on their similarity to the query.

    Visualizations:
        Visualize the TF-IDF vectors in lower dimensions using techniques like t-SNE or PCA.
        Explore the distribution of documents in the vector space.



In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.feature import Normalizer

# Normalize the vectors
normalizer = Normalizer(inputCol="tfidf_features", outputCol="normFeatures")
data = normalizer.transform(final_tfidf_vector)

# Get the first document's features
first_doc_features = data.first().tfidf_features
def cosine_similarity(v):
    dot_product = float(first_doc_features.dot(v))
    norm_product = float(first_doc_features.norm(2)) * float(v.norm(2))
    return float(dot_product) / float(norm_product)


cosine_similarity_udf = udf(cosine_similarity, FloatType())

# Compute the cosine similarity and add it as a new column
data = data.withColumn("cosine_sim", cosine_similarity_udf(col("normFeatures")))

# Show the top 5 documents
#data.sort(col("cosine_sim").desc()).select('id', 'cosine_sim').show(5)

In [34]:
data = data.withColumn("cosine_sim2", cos_sim_udf(col("normFeatures")))
data.sort(col("cosine_sim").desc()).select('id', 'cosine_sim').show(5)

24/03/20 13:05:46 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


+--------------+----------+
|            id|cosine_sim|
+--------------+----------+
| CVE-2020-0003|       1.0|
|CVE-2020-18475| 0.9601347|
|CVE-2020-15007|0.95723075|
|CVE-2020-36034| 0.9530993|
| CVE-2020-0218| 0.9470317|
+--------------+----------+
only showing top 5 rows



# The results:
## Results from first iteration - No stop words were removed
### CVE-2020-0003 (Search Document)
In onCreate of InstallStart.java, there is a possible package validation bypass due to a time-of-check time-of-use vulnerability. This could lead to local escalation of privilege with no additional execution privileges needed. User interaction is needed for exploitation. Product: Android Versions: Android-8.0 Android ID: A-140195904

### CVE-2020-25162
A XPath injection vulnerability in the B. Braun Melsungen AG SpaceCom Version L81/U61 and earlier, and the Data module compactplus Versions A10 and A11 allows unauthenticated remote attackers to access sensitive information and escalate privileges

### CVE-2020-24721
An issue was discovered in the GAEN (aka Google/Apple Exposure Notifications) protocol through 2020-09-29, as used in COVID-19 applications on Android and iOS. It allows a user to be put in a position where he or she can be coerced into proving or disproving an exposure notification, because of the persistent state of a private framework.

There could be a few reasons why the documents getting aren't as similar as I'd like. Here are some things to consider:

1. **Quality of your data**: The quality and relevance of the documents in your dataset can greatly affect the results of cosine similarity. If the documents in your dataset are not very similar to your target document to begin with, the top results might not seem very similar.

2. **Preprocessing**: How you preprocess your data can also affect the results. This includes things like removing stop words, stemming or lemmatization, and how you handle punctuation and capitalization. You might need to adjust your preprocessing steps to better suit your data.

3. **Vectorization**: The method you use to convert your text data into numerical vectors can also have a big impact. You're currently using TF-IDF, which is a good choice for many applications, but there might be other methods that work better for your specific dataset. You could experiment with other methods like Word2Vec, Doc2Vec, or BERT embeddings.

4. **Dimensionality**: High-dimensional data can be problematic for cosine similarity due to the curse of dimensionality. You might want to try reducing the dimensionality of your data with techniques like PCA or t-SNE.

5. **Thresholding**: You might want to consider applying a threshold to your cosine similarity scores. This means that you only consider documents as "similar" if their cosine similarity score is above a certain threshold.



In [ ]:
spark.stop()